In [ ]:
%matplotlib inline

## Iris introduction course
# 4. Joining Cubes Together

**Learning outcome**: by the end of this section, you will be able to apply Iris functionality to combine multiple Iris cubes into a new larger cube.

**Duration:** 1 hour

**Overview:**<br>
3.1 [Merge](#merge)<br>
3.2 [Concatenate](#concatenate)<br>
3.3 [Summary of the Section](#summary)

## Setup

In [ ]:
import iris

----

## 3.1 Merge<a id='merge'></a>

When Iris loads data it tries to reduce the number of cubes returned by collecting together multiple fields with
shared metadata into a single multidimensional cube. In Iris, this is known as merging.

In order to merge two cubes, they must be identical in everything but a scalar dimension, which goes on to become a new data dimension.

The ``iris.load_raw`` function can be used as a diagnostic tool to identify the individual "fields" that Iris identifies in a given set of filenames before any merge takes place:

In [ ]:
fname = iris.sample_data_path('GloSea4', 'ensemble_008.pp')
raw_cubes = iris.load_raw(fname)

print(len(raw_cubes))

When we look in detail at these cubes, we find that they are identical in every coordinate except for the scalar forecast_period and time coordinates:

In [ ]:
print(raw_cubes[0])
print('--' * 50)
print(raw_cubes[1])

Any CubeList can be merged with the ``merge`` method, and the resulting CubeList from load_raw is no different.
The ``merge`` method *always* returns another CubeList:

In [ ]:
merged_cube, = raw_cubes.merge()
print(merged_cube)

When we look in more detail, we can see that the time coordinate has become a new dimension, as well as gaining another forecast_period auxiliary coordinate:

In [ ]:
print(merged_cube.coord('time'))
print(merged_cube.coord('forecast_period'))

### Identifying merge problems

In order to avoid the Iris merge functionality making often inappropriate assumptions about incoming data, merge is strict with regards to the uniformity of the incoming cubes.

For example, if we load the fields from two ensemble members from the GloSea4 model sample data, we see we have 12 fields before any merge takes place:

In [ ]:
fname = iris.sample_data_path('GloSea4', 'ensemble_00[34].pp')
cubes = iris.load_raw(fname, 'surface_temperature')
print(len(cubes))

If we try to merge these 12 cubes we get 2 cubes rather than one:

In [ ]:
incomplete_cubes = cubes.merge(unique=False)
print(incomplete_cubes)

When we look in more detail at these two cubes, what is different between the two? (Hint: One value changes, another is completely missing)

In [ ]:
print(incomplete_cubes[0])
print('--' * 50)
print(incomplete_cubes[1])

By adding the missing coordinate, we can trigger a merge of the 12 cubes into a single cube, as expected:

In [ ]:
for cube in cubes:
    if not cube.coords('realization'):
        cube.add_aux_coord(iris.coords.DimCoord(np.int32(3),
                                                'realization'))

merged_cubes = cubes.merge()
print(merged_cubes)

Iris includes functionality to simplify the identification process for causes of failed merges. The ``merge_cube`` method of a CubeList expects the list of cubes to contain only cubes that can be merged to produce a single cube. If they do not merge to a single cube, a descriptive exception will be raised. For instance:

```
   >>> cubes.merge_cube()
   Traceback (most recent call last):
     File "<stdin>", line 1, in <module>
     ...
   iris.exceptions.MergeError: failed to merge into a single cube.
     Coordinates in cube.aux_coords (scalar) differ: realization.
```

## 3.2 Concatenate<a id='concatenate'></a>

We have seen that merge combines a list of cubes with a common scalar coordinate to produce a single cube with a new dimension created from these scalar values.

But what happens if you try to combine cubes along a common dimension?

In [ ]:
fname = iris.sample_data_path('A1B_north_america.nc')
cube = iris.load_cube(fname)

cube_1 = cube[:10]
cube_2 = cube[10:20]
cubes = iris.cube.CubeList([cube_1, cube_2])
print(cubes)

These cubes should be able to be merged; after all, they have both come from the same original cube!

In [ ]:
print(cubes.merge())

Merge cannot be used to combine common non-scalar coordinates. Instead we must use concatenate, which joins together ("concatenates") common non-scalar coordinates to produce a single cube with the common dimension extended:

In [ ]:
print(cubes.concatenate())

As with merge, Iris contains functionality to simplify the identification process for causes of failed concatenations. The ``concatenate_cube`` method of a CubeList expects the list of cubes to contain only cubes that can be concatenated to produce a single cube. If they do not concatenate to a single cube, a descriptive error will be raised. For instance:

```
    >>> print cubes.concatenate_cube()
    Traceback (most recent call last):
      ...
    iris.exceptions.ConcatenateError: failed to concatenate into a single cube.
      Scalar coordinates differ: forecast_reference_time, height != forecast_reference_time
```

### Exercise 3 : Solving Merge problems

The following exercise is designed to give you experience of solving issues that prevent a merge from taking place.
The output from ``merge_cube`` is included to help with identification, and once a fix has been identified, ``raw_cubes.merge()`` should result in a CubeList containing a single cube:

The first exercise is completed below:

1\. Identify and resolve the issue preventing the merge of ``air_potential_temperature`` cubes from ``resources/merge_exercise.1.*.nc``.

    >>> raw_cubes = iris.load_raw('resources/merge_exercise.1.*.nc', 'air_potential_temperature')
    >>> raw_cubes.merge_cube()
    Traceback (most recent call last):
    ...
    iris.exceptions.MergeError: failed to merge into a single cube.
      cube.attributes keys differ: 'History'


In [ ]:
raw_cubes = iris.load_raw('resources/merge_exercise.1.*.nc', 'air_potential_temperature')

# Print the attributes, clearly one is different.
for cube in raw_cubes:
    print(cube.attributes)

# Remove the history attribute from the first cube.
del raw_cubes[0].attributes['History']

# Check that this has meant that a merge now results in a single cube.
print('--' * 50)
print(raw_cubes.merge())

2\. Identify and resolve the issue preventing the merge of ``air_potential_temperature`` cubes from ``resources/merge_exercise.5.*.nc`` (hint: can these cubes be merged?).

    >>> raw_cubes = iris.load_raw('resources/merge_exercise.5.*.nc', 'air_potential_temperature')
    >>> raw_cubes.merge_cube()
    Traceback (most recent call last):
    ...
    iris.exceptions.MergeError: failed to merge into a single cube.
      Coordinates in cube.dim_coords differ: time.

## 3.3 Section Summary : Cube Control<a id='summary'></a>

In this section we learnt:
* Merging is used to join cubes into a larger combined dataset
* Concatenation is a similar operation, used in slightly different circumstances to merging.
